In [49]:
import numpy as np
import pandas as pd

In [50]:
def save_win_dataframe_to_csv(dataframe, filename):
    df = pd.DataFrame(dataframe)
    df.to_csv(filename, index=False)

In [51]:
def save_lose_dataframe_to_csv(dataframe, filename):
    df = pd.DataFrame(dataframe)
    columns_to_exclude_index = [0, 1, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 26, 34]  
    columns_to_multiply_by_minus_1_index = [col_idx for col_idx in range(len(df.columns)) if col_idx not in columns_to_exclude_index]
    df.iloc[:, columns_to_multiply_by_minus_1_index] = df.iloc[:, columns_to_multiply_by_minus_1_index] * -1
    df.to_csv(filename, index=False)

In [73]:
data1 = pd.read_csv("Dataset/win/Platinum_I.csv")
data2 = pd.read_csv("Dataset/win/Platinum_II.csv")
data3 = pd.read_csv("Dataset/win/Platinum_III.csv")
data4 = pd.read_csv("Dataset/win/Platinum_IV.csv")

#티어별로 2500개씩 슬라이싱
data1 = data1.drop(data1.index[2500:])
data2 = data2.drop(data2.index[2500:])
data3 = data3.drop(data3.index[2500:])
data4 = data4.drop(data4.index[2500:])




In [66]:
print(data1)

            matchId  queueId  Diff_LV  Diff_CS  Diff_jglCS  Diff-K  \
0     KR_6728675184      420       -3      -12         -10     -10   
1     KR_6755780484      440        4       81          28       9   
2     KR_6768062927      440       -3       24         -23      -4   
3     KR_6759156095      420        1       67          12       5   
4     KR_6605744824      440        1      -12         -28       8   
...             ...      ...      ...      ...         ...     ...   
2496  KR_6530673429      420       -1       22         -20      -7   
2497  KR_6683153161      420        1       21          21      -9   
2498  KR_6764933526      420        1       60          32      11   
2499  KR_6758172560      420       -2        7         -34       6   
2500  KR_6726879839      420       -1      -23          -7      -4   

      Diff-K-top  K-WIN-top  K-LOSE-top  Diff-K-jug  ...  \
0              1          3           2          -3  ...   
1              1          1           0

In [74]:
windata = pd.concat([data1,data2,data3,data4])
windata = windata.drop_duplicates(subset=['matchId'], keep='first')  # 또는 keep='last'

In [75]:
save_win_dataframe_to_csv(windata,"Dataset/win/Platinum_win.csv")

In [55]:
save_lose_dataframe_to_csv(windata,"Dataset/lose/Gold_lose_concat_finalsss.csv")

In [56]:
losedata = pd.read_csv("Dataset/lose/Gold_lose_concat_finalsss.csv")

In [58]:
compare = losedata[losedata['matchId'].duplicated(keep=False)]
print(compare['matchId'])

Series([], Name: matchId, dtype: object)


In [59]:

data = pd.concat([windata, losedata])

In [48]:
print(data['Diff-A'].mean())

0.0


In [57]:
data['Diff-A'] = data['Diff-A']*-1


In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# df = pd.read_csv('Grandmaster_concat.csv')


#data['result'] = data['result'].replace({-1: 0})



features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
#df[features]을 통해 특정 칼럼들로 학습 데이터로 사용한다. 그리고 df['result'] 이 부분을 통해 각 게임에 대한 승패 결과를 교육(?)
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=12, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1], got [ 1. nan]

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = RandomForestClassifier(n_estimators=120, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.9151076597246735


In [10]:
print(windata["result"])

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
2647   -1
2648   -1
2649   -1
2650   -1
2651   -1
Name: result, Length: 13167, dtype: int64


In [29]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = LGBMClassifier(n_estimators=400)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

[LightGBM] [Info] Number of positive: 9888, number of negative: 9862
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001223 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 19750, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500658 -> initscore=0.002633
[LightGBM] [Info] Start training from score 0.002633
정확도: 0.7777946537059538


In [74]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = CatBoostClassifier(iterations=100, random_state=123)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

Learning rate set to 0.285308
0:	learn: 0.5164657	total: 154ms	remaining: 15.3s
1:	learn: 0.3980603	total: 163ms	remaining: 8s
2:	learn: 0.3419642	total: 172ms	remaining: 5.57s
3:	learn: 0.3175816	total: 182ms	remaining: 4.37s
4:	learn: 0.2918697	total: 191ms	remaining: 3.63s
5:	learn: 0.2765550	total: 203ms	remaining: 3.18s
6:	learn: 0.2672256	total: 214ms	remaining: 2.84s
7:	learn: 0.2534970	total: 223ms	remaining: 2.56s
8:	learn: 0.2437030	total: 234ms	remaining: 2.37s
9:	learn: 0.2411426	total: 244ms	remaining: 2.19s
10:	learn: 0.2288217	total: 253ms	remaining: 2.05s
11:	learn: 0.2215912	total: 263ms	remaining: 1.93s
12:	learn: 0.2114128	total: 272ms	remaining: 1.82s
13:	learn: 0.2058374	total: 282ms	remaining: 1.73s
14:	learn: 0.1977432	total: 293ms	remaining: 1.66s
15:	learn: 0.1950502	total: 303ms	remaining: 1.59s
16:	learn: 0.1881008	total: 315ms	remaining: 1.54s
17:	learn: 0.1854144	total: 325ms	remaining: 1.48s
18:	learn: 0.1795411	total: 335ms	remaining: 1.43s
19:	learn: 0.1

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

data['result'] = data['result'].replace({-1: 0})


features = ['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-K-top', 'Diff-K-jug', 'Diff-K-mid', 'Diff-K-ad',
            'Diff-K-sup', 'invadeKill', 'Diff-A', 'Diff_WARDplaced', 'Diff-ControlWARDplaced', 'Diff_WARDkill',
            'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 'Diff_FirstBLOOD']


X_train, X_test, y_train, y_test = train_test_split(data[features], data['result'], test_size=0.25, random_state=32)
model = ExtraTreesClassifier(n_estimators=100, random_state=32)#n_estimators는 의사 결정 트리의 수. 값을 높게 설정 할 수록 정확도가 좋다고 한다.
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

정확도: 0.7952612393681653
